In [17]:
import pandas as pd
import requests
import time

In [18]:

gsc_data = pd.read_csv('GSC+.csv', converters={'HPO_codes': eval, 'starts': eval, 'ends': eval, 'HPO_symptoms': eval})

In [19]:
# Translate English to Spanish and keep both in final document 
def translate_english_spanish(doc_id,text, symptoms, hpo_codes):
    
    #define HTTP header, param and endpoint
    endpoint = 'https://f29bio-dev.northeurope.cloudapp.azure.com/api/Translation/document/translate-hack'
    headers = {'accept': 'text/plain','Content-Type': 'application/json',}
    params = (('lan', 'es'),)
    
    #Translate the description of the condition (symptom(s))
    data = '{"language_source":"en","segments":[{"id":"'+ str(doc_id)+ '","source":"'+ text +'"}]}'
    response = requests.post(endpoint, headers=headers, params=params, data=data)
    
    if response.status_code == 200:
        
        output = response.json()
        file = output['segments'][0]['id']
        translation = output['segments'][0]['target']
        translated_symptoms = []
        
        #translate the symtoms themselves
        for symptom in symptoms:
            data = '{"language_source":"en","segments":[{"id":"'+ str(doc_id)+ '","source":"'+ symptom +'"}]}'
            response = requests.post(endpoint, headers=headers, params=params, data=data) 
            if response.status_code == 200:
                output = response.json()
                translated_symptoms.append(output['segments'][0]['target'])
            else:
                return{}
            
        #Return all the data to mimic original file although, it has         
        return {
            'Archivo': file,
            'Texto_espanol': translation,
            'Texto_inglés': text,
            'HPO_síntomas': translated_symptoms,
            'HPO_código': hpo_codes
        }
    else:
        return{}

In [21]:
translation_rows = []
for row in gsc_data.itertuples():
    translation_output = translate_english_spanish(row[1], row[2], row[3], row[4])
    translation_rows.append(translation_output)
    if (row[0] + 1) % 10 == 0:
        print('Fetched {} rows'.format(row[0] + 1))
        time.sleep(1)
translation_df = pd.DataFrame(translation_rows)

Fetched 10 rows
Fetched 20 rows
Fetched 30 rows
Fetched 40 rows
Fetched 50 rows
Fetched 60 rows
Fetched 70 rows
Fetched 80 rows
Fetched 90 rows
Fetched 100 rows
Fetched 110 rows
Fetched 120 rows
Fetched 130 rows
Fetched 140 rows
Fetched 150 rows
Fetched 160 rows
Fetched 170 rows
Fetched 180 rows
Fetched 190 rows
Fetched 200 rows
Fetched 210 rows
Fetched 220 rows


In [22]:
translation_df.head(10)

,Archivo,Texto_espanol,Texto_inglés,HPO_síntomas,HPO_código
0,1003450,En cinco miembros de una familia italiana se r...,A syndrome of brachydactyly (absence of some m...,"[brachydactyly, ausencia de algunas falanges m...","[0001156, 0009881, 0001798, 0001792, 0100264, ..."
1,10051003,El síndrome de Townes-Brock (TBS) es un trasto...,Townes-Brocks syndrome (TBS) is an autosomal d...,"[trastorno autosómico dominante., autosómica d...","[0000006, 0000006, 0000006, 0003828, 0003813, ..."
2,10066029,El síndrome de carcinoma basocelular nevoide (...,Nevoid basal cell carcinoma syndrome (NBCCS) i...,"[carcinoma basocelular., autosómica dominante....","[0002671, 0000006, 0000006, 0000006, 0000006, ..."
3,10196695,El síndrome de Angélica (EA) es un trastorno d...,Angelman syndrome (AS) is a neurodevelopmental...,"[trastorno del neurodesarrollo, síndrome del g...","[0000707, 0001466]"
4,10417280,El síndrome de Prader-Willi (PWS) y el síndrom...,Prader-Willi syndrome (PWS) and Angelman syndr...,"[trastornos neuroconductuales, de Novo]","[0000708, 0003745]"
5,10440824,Presentamos la descripción de una niña de 14 m...,We describe a 14-month-old girl with unilatera...,"[colesteatoma, anomalías del nervio facial., m...","[0009797, 0010827, 0009794, 0009794, 0000598, ..."
6,10482951,Las características clínicas del síndrome de A...,The clinical features of Angelman syndrome (AS...,"[retraso mental severo., retraso mental., retr...","[0002187, 0001249, 0001249, 0005484, 0000252, ..."
7,NaN,NaN,NaN,NaN,NaN
8,10593995,El síndrome de poplítea pterigión (PPS) es un ...,Popliteal pterygium syndrome (PPS) is a rare a...,"[Pterigión poplítea., pterigión, trastorno aut...","[0009756, 0001059, 0000006, 0000006, 0000006, ..."
9,10631923,El síndrome de Townes-Brock (TBS) ha sido reco...,Townes-Brocks syndrome (TBS) has been recogniz...,"[dominante, hereditario dominante, ano imperfo...","[0000006, 0000006, 0002023, 0001199, 0000598, ..."


In [24]:
#Drop all Nan values
translation_df.dropna()

,Archivo,Texto_espanol,Texto_inglés,HPO_síntomas,HPO_código
0,1003450,En cinco miembros de una familia italiana se r...,A syndrome of brachydactyly (absence of some m...,"[brachydactyly, ausencia de algunas falanges m...","[0001156, 0009881, 0001798, 0001792, 0100264, ..."
1,10051003,El síndrome de Townes-Brock (TBS) es un trasto...,Townes-Brocks syndrome (TBS) is an autosomal d...,"[trastorno autosómico dominante., autosómica d...","[0000006, 0000006, 0000006, 0003828, 0003813, ..."
2,10066029,El síndrome de carcinoma basocelular nevoide (...,Nevoid basal cell carcinoma syndrome (NBCCS) i...,"[carcinoma basocelular., autosómica dominante....","[0002671, 0000006, 0000006, 0000006, 0000006, ..."
3,10196695,El síndrome de Angélica (EA) es un trastorno d...,Angelman syndrome (AS) is a neurodevelopmental...,"[trastorno del neurodesarrollo, síndrome del g...","[0000707, 0001466]"
4,10417280,El síndrome de Prader-Willi (PWS) y el síndrom...,Prader-Willi syndrome (PWS) and Angelman syndr...,"[trastornos neuroconductuales, de Novo]","[0000708, 0003745]"
...,...,...,...,...,...
223,9831341,Hemos descrito previamente a un paciente con u...,We had previously described a patient with an ...,"[crecimiento, hiperfagia, obesidad, crecimiento]","[0001548, 0002591, 0001513, 0001548]"
224,9863591,Los pacientes que presentan schwanomas vestibu...,Patients who present with unilateral vestibula...,"[schwanomas vestibulares unilaterales., schwan...","[0009590, 0009588, 0100008, 0003593, 0003593, ..."
225,9931336,Se han identificado mutaciones del gen parchea...,Mutations of the human Patched gene ( PTCH ) h...,"[carcinoma basocelular., esporádicos, carcinom...","[0002671, 0003745, 0002671, 0002885, 0002671, ..."
226,9949213,Los elementos de repetición de copia baja se a...,"Transcribed, low-copy repeat elements are asso...","[anomalías del desarrollo, letalidad juvenil.]","[0001263, 0004149]"


In [43]:
translation_df.head(10)

,Archivo,Texto_espanol,Texto_inglés,HPO_síntomas,HPO_código
0,1003450,En cinco miembros de una familia italiana se r...,A syndrome of brachydactyly (absence of some m...,"[brachydactyly, ausencia de algunas falanges m...","[0001156, 0009881, 0001798, 0001792, 0100264, ..."
1,10051003,El síndrome de Townes-Brock (TBS) es un trasto...,Townes-Brocks syndrome (TBS) is an autosomal d...,"[trastorno autosómico dominante., autosómica d...","[0000006, 0000006, 0000006, 0003828, 0003813, ..."
2,10066029,El síndrome de carcinoma basocelular nevoide (...,Nevoid basal cell carcinoma syndrome (NBCCS) i...,"[carcinoma basocelular., autosómica dominante....","[0002671, 0000006, 0000006, 0000006, 0000006, ..."
3,10196695,El síndrome de Angélica (EA) es un trastorno d...,Angelman syndrome (AS) is a neurodevelopmental...,"[trastorno del neurodesarrollo, síndrome del g...","[0000707, 0001466]"
4,10417280,El síndrome de Prader-Willi (PWS) y el síndrom...,Prader-Willi syndrome (PWS) and Angelman syndr...,"[trastornos neuroconductuales, de Novo]","[0000708, 0003745]"
5,10440824,Presentamos la descripción de una niña de 14 m...,We describe a 14-month-old girl with unilatera...,"[colesteatoma, anomalías del nervio facial., m...","[0009797, 0010827, 0009794, 0009794, 0000598, ..."
6,10482951,Las características clínicas del síndrome de A...,The clinical features of Angelman syndrome (AS...,"[retraso mental severo., retraso mental., retr...","[0002187, 0001249, 0001249, 0005484, 0000252, ..."
7,NaN,NaN,NaN,NaN,NaN
8,10593995,El síndrome de poplítea pterigión (PPS) es un ...,Popliteal pterygium syndrome (PPS) is a rare a...,"[Pterigión poplítea., pterigión, trastorno aut...","[0009756, 0001059, 0000006, 0000006, 0000006, ..."
9,10631923,El síndrome de Townes-Brock (TBS) ha sido reco...,Townes-Brocks syndrome (TBS) has been recogniz...,"[dominante, hereditario dominante, ano imperfo...","[0000006, 0000006, 0002023, 0001199, 0000598, ..."


In [ ]:
#Save to file
translation_df.to_csv('GSC+_es_and_en.csv')